<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-demos/haystack_rag_services_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook showcases a novel application of Qdrant vector DB in the upcoming Haystack OpenAPI service-based Retriever-Augmented Generation (RAG) system. Given a user query, we retrieve the corresponding OpenAPI specification from Qdrant vector DB. The retrieved service specification is then seamlessly injected into the Haystack pipeline, enabling the dynamic invocation of selected API services. This innovative approach, integrating any OpenAPI spec service with LLMs, opens a new era for RAG systems to generate contextually rich responses from any openapi service.

## 1. Setup
This notebook demos Haystack 2.x service based RAG, using two openapi services: GitHub's compare_branches and SerperDev search API.

Let's install necessary libraries and import key modules to build the foundation for the subsequent steps.

In [1]:
!pip uninstall -y llmx

Found existing installation: llmx 0.0.15a0
Uninstalling llmx-0.0.15a0:
  Successfully uninstalled llmx-0.0.15a0


In [2]:
!pip install -q "grpcio-tools==1.42" sentence-transformers openapi3 jsonref

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00


In [3]:
!pip install -q git+https://github.com/vblagoje/qdrant-haystack.git@haystack-v2 git+https://github.com/deepset-ai/haystack.git@openapi_container_v5

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [4]:
import getpass
import os
import json
import requests
from qdrant_haystack import QdrantDocumentStore
from qdrant_haystack.retriever import QdrantRetriever

from haystack import Pipeline
from haystack.components.generators.utils import default_streaming_callback
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.converters import OpenAPIServiceToFunctions
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import GPTChatGenerator
from haystack.components.connectors import OpenAPIServiceConnector
from haystack.dataclasses import ChatMessage

## 2. Setting up Qdrant vector DB, create indexing pipeline

In [5]:
document_store = QdrantDocumentStore(
    path="./qdrant_v1",
    index="Document",
    embedding_dim=768,
    recreate_index=True
  )

In [6]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("spec_to_functions", OpenAPIServiceToFunctions())
indexing_pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
indexing_pipeline.add_component("writer", DocumentWriter(document_store=document_store))
indexing_pipeline.connect(connect_from="spec_to_functions", connect_to="embedder")
indexing_pipeline.connect(connect_from="embedder", connect_to="writer")

## 3. Index two openapi specs (Github and SerperDev), along with system prompts

In [7]:
indexing_pipeline.run(data={"sources":["https://bit.ly/3tdRUM0", "https://bit.ly/3NIJqnd"],
                            "system_messages":[requests.get("https://bit.ly/48eN0ND").text,
                                               requests.get("https://bit.ly/3TdHsyB").text]})

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100it [00:00, 2319.32it/s]           


{'writer': {'documents_written': 2}}

In [8]:
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model_name_or_path="sentence-transformers/all-mpnet-base-v2"))
retrieval_pipeline.add_component("retriever", QdrantRetriever(document_store=document_store))
retrieval_pipeline.connect(connect_from="embedder", connect_to="retriever")

## 4. API keys, set up simple authentication mechanism

In [9]:
llm_api_key = getpass.getpass("Enter LLM provider api key:")


Enter LLM provider api key:··········


In [10]:
serper_dev_key = getpass.getpass("Enter serperdev api key:")
services_auth = {"SerperDev":serper_dev_key}

Enter serperdev api key:··········


## 5. Retrieval step - service invocation

Based on the prompt, retrieve the right service spec from Qdrant, inject spec and system prompt into the pipeline

In [11]:
invoke_service_pipe = Pipeline()
invoke_service_pipe.add_component("functions_llm", GPTChatGenerator(api_key=llm_api_key, model_name="gpt-3.5-turbo-0613"))
invoke_service_pipe.add_component("openapi_container", OpenAPIServiceConnector(services_auth))
invoke_service_pipe.connect("functions_llm.replies", "openapi_container.messages")

In [12]:
user_prompt = "Search web and tell me why was Sam Altman ousted from OpenAI"
#user_prompt = "Compare branches main and openapi_container_v5, in project deepset-ai, repo haystack"

In [13]:
top_k = 1
top_k_result = retrieval_pipeline.run(data={"text": user_prompt, "top_k": top_k})
top_k_docs = top_k_result["retriever"]["documents"][:top_k]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
top_1_document = top_k_docs[0]
openai_functions_definition = json.loads(top_1_document.content)
openapi_spec = top_1_document.meta["spec"]

service_response = invoke_service_pipe.run(data={"messages":[ChatMessage.from_user(user_prompt)],
                                                 "generation_kwargs": {"functions": [openai_functions_definition]},
                                                 "service_openapi_spec": openapi_spec})

## 6. Generate LLM response

Inject service response into LLM context, pair it with system prompt

In [15]:
gen_pipe = Pipeline()
llm = GPTChatGenerator(api_key=llm_api_key, model_name="gpt-4-1106-preview", streaming_callback=default_streaming_callback)
gen_pipe.add_component("llm", llm)

github_pr_prompt_messages = [ChatMessage.from_system(top_1_document.meta["system_message"])] + service_response["openapi_container"]["service_response"]
final_result = gen_pipe.run(data={"messages": github_pr_prompt_messages})

Sam Altman was ousted from his position as CEO of OpenAI by the company's board. The decision, described as "shocking," took place on the Friday before Thanksgiving. The board provided a statement indicating that the reason for his dismissal was related to Altman not being "consistently candid" with the directors. However, in a subsequent turn of events, Altman was reinstated as CEO of OpenAI just days after his ouster, which capped off frenzied discussions about the future of the startup. The specifics behind this reversal are not detailed in the snippets provided.

## Thank you, questions?

<a href="www.qr-code-generator.com/" border="0" style="cursor:default" rel="nofollow"><img src="https://chart.googleapis.com/chart?cht=qr&chl=https%3A%2F%2Fgithub.com%2Fvblagoje%2Fnotebooks%2Fblob%2Fmain%2Fhaystack2x-demos%2Fhaystack_rag_services_demo.ipynb&chs=180x180&choe=UTF-8&chld=L|2"></a>

## Links:
- https://github.com/deepset-ai/haystack/
- https://haystack.deepset.ai/community
- https://haystack.deepset.ai/advent-of-haystack
- https://x.com/vladblagoje